In [10]:
!unzip carletonbuildingsreviewed.zip

Archive:  carletonbuildingsreviewed.zip
  inflating: All_Images_500x500/AA/030_1.jpg  
  inflating: All_Images_500x500/AA/030_2.jpg  
  inflating: All_Images_500x500/AA/030_3.jpg  
  inflating: All_Images_500x500/AA/030_4.jpg  
  inflating: All_Images_500x500/AA/030_5.jpg  
  inflating: All_Images_500x500/AA/081_1.jpg  
  inflating: All_Images_500x500/AA/081_2.jpg  
  inflating: All_Images_500x500/AA/081_3.jpg  
  inflating: All_Images_500x500/AA/081_4.jpg  
  inflating: All_Images_500x500/AA/081_5.jpg  
  inflating: All_Images_500x500/AA/120-1.jpg  
  inflating: All_Images_500x500/AA/120-2.jpg  
  inflating: All_Images_500x500/AA/120-3.jpg  
  inflating: All_Images_500x500/AA/120-4.jpg  
  inflating: All_Images_500x500/AA/120-5.jpg  
  inflating: All_Images_500x500/AA/120-6.jpg  
  inflating: All_Images_500x500/AA/128-1.jpg  
  inflating: All_Images_500x500/AA/128-2.jpg  
  inflating: All_Images_500x500/AA/128-3.jpg  
  inflating: All_Images_500x500/AA/128-4.jpg  
  inflating: All_Ima

In [5]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras
import tensorflow as tf

nasnet = keras.applications.nasnet.NASNetLarge(include_top=False, weights='imagenet')
nasnet.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_260 (Activation)     (None, 165, 165, 96) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [6]:
for layer in nasnet.layers[:600]:
    layer.trainable = False
nasnet.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_260 (Activation)     (None, 165, 165, 96) 0           stem_bn1[0][0]                   
_____________________________________________________________________________________________

In [0]:
model = keras.models.Sequential()
model.add(nasnet)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(16, activation='softmax'))

In [0]:
model.compile(keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.3)

train_generator = train_datagen.flow_from_directory(
        './All_Images_500x500',
        target_size=(331, 331),
        batch_size=32,
        class_mode='categorical')

Found 3302 images belonging to 16 classes.


In [18]:
model.fit_generator(
        train_generator,
        steps_per_epoch=50,
        epochs=35)

Epoch 1/5
50/50 [==============================] - 65s 1s/step - loss: 0.3658 - accuracy: 0.9333
Epoch 2/5
50/50 [==============================] - 65s 1s/step - loss: 0.4116 - accuracy: 0.9075
Epoch 3/5
50/50 [==============================] - 63s 1s/step - loss: 0.2970 - accuracy: 0.9396
Epoch 4/5
50/50 [==============================] - 64s 1s/step - loss: 0.3819 - accuracy: 0.9350
Epoch 5/5
50/50 [==============================] - 63s 1s/step - loss: 0.2681 - accuracy: 0.9333


In [20]:
model.pop()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
flatten_1 (Flatten)          (None, 487872)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               62447744  
Total params: 147,364,562
Trainable params: 132,746,672
Non-trainable params: 14,617,890
_________________________________________________________________


In [0]:
model.save("encoder.h5")

In [27]:
!zip -r ./model.zip ./encoder.h5

  adding: encoder.h5 (deflated 25%)
